In [1]:
import numpy as np
import pandas as pd
import struct
import cv2 as cv
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import scale

In [3]:
with open('../scenes/sample.dat', 'rb') as f:
    fileContent = f.read()
    width, height, spp, SL = struct.unpack("QQQQ", fileContent[:32])
    samples = struct.unpack("f" * ((len(fileContent) - 32) // 4), fileContent[32:])
    raw = np.array(samples)
    samples = np.reshape(raw, (width, height, spp, SL))
print(width, height, spp, SL)
print(samples[10,5,4,:])    

100 100 8 28
[ 5.82861328e+00  1.05459538e+01  2.56491732e-02  2.56491732e-02
  4.10386808e-02 -4.00000000e+02  2.95105316e+02 -1.33042297e+01
  0.00000000e+00  0.00000000e+00  0.00000000e+00  5.00000000e-01
  5.00000000e-01  8.00000012e-01  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.25136793e-01
  3.10504138e-01  5.03994346e-01  0.00000000e+00  0.00000000e+00]


In [30]:
def preprocess(x, y, b, M):
    # Sample neighbors based on gaussian distribution
    indexN = np.random.multivariate_normal(mean=[x, y], cov=(b/4) * np.identity(2), size=M-spp)
    indexN = np.clip(indexN, 0, 100)
    indexN = np.around(indexN)
    randSampleInPixel = np.random.randint(7, size=(M-spp, 1))
    indexN = np.concatenate((indexN,randSampleInPixel), axis = 1)
    sampleMapper = lambda v: samples[int(v[0]), int(v[1]), int(v[2]),:]
    N = np.apply_along_axis(sampleMapper, 1, indexN)
    # TODO: cluster samples
    # Standardize neighbors
    N = scale(N)
    return N

In [31]:
boxSizes = [55, 35, 17, 7]
for t in range(4):
    b = boxSizes[t]
    maxNumOfSamples = (b * b * spp) // 2
    for i in range(height):
        for j in range(width):
            N = preprocess(i, j, b, maxNumOfSamples)

KeyboardInterrupt: 